In [1]:
import numpy as np

# --- Conversion functions as provided ---

def augment(v):
    # v is (N, 3); this returns an (N, 4) array with ones in the last column.
    return np.concatenate([v, np.ones_like(v[:, :1])], axis=-1)

def augmentT(v):
    # v is (M, N); this returns an (M+1, N) array with an extra row of ones.
    return np.concatenate([v, np.ones_like(v[:1])], axis=0)

def point2image(vertices, viewmat, projection_matrix, cam_pos, eps=np.finfo(np.float32).eps):
    """
    Convert a set of 3D vertices to 2D pixel-space coordinates plus the per-vertex inverse distance.
    
    vertices: (N,3)
    viewmat: (4,4)
    projection_matrix: (3,3)
    cam_pos: (3,)
    
    Returns an (N, 3) array where each row is [x, y, inv_distance].
    """
    # Transform vertices into camera space (homogeneous coordinates)
    cam_space_homo = viewmat @ augment(vertices).T  # shape (4, N)
    # Perform perspective division on the first two coordinates using the z coordinate.
    cam_space_nohomo = cam_space_homo[:2] / (np.abs(cam_space_homo[2:3]) + 1e-10)
    # Map to pixel space (here using an identity projection_matrix)
    pixel_space = projection_matrix @ augmentT(cam_space_nohomo)  # shape (3, N)
    # Compute the inverse of the distance from each vertex to the camera.
    inv_distance = 1 / np.clip(cam_space_homo[2:3].T, eps, None)
    # inv_distance = 1 / np.sqrt(
    #     np.clip(np.sum((vertices - cam_pos.reshape(1, 3))**2, axis=1, keepdims=True), eps, None)
    # )
    # Concatenate the 2D positions with the inverse distance.
    return np.concatenate([pixel_space[:2].T, inv_distance], axis=1)

# --- 2D Ray–Triangle Intersection (using barycentrics) ---

def ray_triangle_intersect_2D(pixelPos, t0, t1, t2, w0, w1, w2):
    """
    Given a 2D pixel position and a 2D triangle (with w=1/distance at each vertex),
    compute the perspective-correct distance along the ray.
    
    Returns (hit, distance) where hit is True if the pixel is inside the triangle.
    """
    # Ensure inputs are numpy arrays.
    pixelPos = np.asarray(pixelPos, dtype=np.float32)
    t0 = np.asarray(t0, dtype=np.float32)
    t1 = np.asarray(t1, dtype=np.float32)
    t2 = np.asarray(t2, dtype=np.float32)
    
    # Compute edge vectors from t0.
    e0 = t1 - t0
    e1 = t2 - t0
    v  = pixelPos - t0

    # Compute dot products.
    dot00 = np.dot(e0, e0)
    dot01 = np.dot(e0, e1)
    dot02 = np.dot(e0, v)
    dot11 = np.dot(e1, e1)
    dot12 = np.dot(e1, v)

    # Compute barycentrics.
    denom = dot00 * dot11 - dot01 * dot01
    alpha = (dot11 * dot02 - dot01 * dot12) / denom
    beta  = (dot00 * dot12 - dot01 * dot02) / denom
    gamma = 1.0 - alpha - beta

    # If any barycentric coordinate is negative, the pixel is outside the triangle.
    if alpha < 0.0 or beta < 0.0 or gamma < 0.0:
        return False, None

    # Interpolate the reciprocal distance.
    reciprocalDist = alpha * w1 + beta * w2 + gamma * w0
    if reciprocalDist <= 0.0:
        return False, None

    # Final perspective-correct distance.
    distance = 1.0 / reciprocalDist
    return True, distance

# --- 3D Ray–Triangle Intersection (Möller–Trumbore algorithm) ---

def safe_div(a, b, eps=1e-8):
    return a / b if abs(b) >= eps else 0.0

def ray_triangle_intersect3D(ray_origin, ray_vector, tri_a, tri_b, tri_c):
    """
    Compute the 3D ray-triangle intersection using the Möller–Trumbore algorithm.
    
    Returns (hit, t) where t is the ray parameter at the intersection.
    """
    ray_origin = np.asarray(ray_origin, dtype=np.float32)
    ray_vector = np.asarray(ray_vector, dtype=np.float32)
    tri_a = np.asarray(tri_a, dtype=np.float32)
    tri_b = np.asarray(tri_b, dtype=np.float32)
    tri_c = np.asarray(tri_c, dtype=np.float32)

    edge1 = tri_b - tri_a
    edge2 = tri_c - tri_a
    ray_cross_e2 = np.cross(ray_vector, edge2)
    det = np.dot(edge1, ray_cross_e2)
    inv_det = safe_div(1.0, det)
    s = ray_origin - tri_a
    u = inv_det * np.dot(s, ray_cross_e2)
    if u < 0 or u > 1:
        return False, None
    s_cross_e1 = np.cross(s, edge1)
    v = inv_det * np.dot(ray_vector, s_cross_e1)
    if v < 0 or (u + v) > 1:
        return False, None
    t = inv_det * np.dot(edge2, s_cross_e1)
    return True, t

# --- Testing the 2D and 3D intersections ---

if __name__ == "__main__":
    # Camera and projection setup.
    cam_pos = np.array([0.0, 0.0, 0.0], dtype=np.float32)
    viewmat = np.eye(4, dtype=np.float32)  # Identity view matrix.
    projection_matrix = np.eye(3, dtype=np.float32)  # Identity projection (i.e. no additional mapping).

    # Define a 3D triangle (in front of the camera at z = 5).
    vertices = np.array([
        [1.0,  1.0,  5.0],
        [-1.0, 1.0,  5.0],
        [0.0,  -1.0, 5.0]
    ], dtype=np.float32)

    # Convert the 3D triangle to a 2D representation.
    pts_2d = point2image(vertices, viewmat, projection_matrix, cam_pos)
    # pts_2d is (3, 3): each row is [x, y, inv_distance]
    
    print("2D triangle vertices (pixel coordinates and inv_distance):")
    print(pts_2d)
    
    # Choose a test pixel: for example, the barycenter of the 2D triangle.
    pixel_pos = np.mean(pts_2d[:, :2], axis=0)
    print("\nTest pixel (barycenter in 2D):", pixel_pos)

    # --- 2D Intersection ---
    hit2D, depth = ray_triangle_intersect_2D(
        pixel_pos,
        pts_2d[0, :2],
        pts_2d[1, :2],
        pts_2d[2, :2],
        pts_2d[0, 2],
        pts_2d[1, 2],
        pts_2d[2, 2]
    )
    print("\n2D Intersection:")
    print("  Hit:", hit2D, "Distance:", distance2D)

    # --- 3D Intersection ---
    # In our simplified setup, the conversion from 3D to 2D is:
    #   pixel = [x/z, y/z]
    # so we can reconstruct a ray direction as [pixel_x, pixel_y, 1] (then normalize).
    ray_dir = np.array([pixel_pos[0], pixel_pos[1], 1.0], dtype=np.float32)
    ray_dir /= np.linalg.norm(ray_dir)
    print("\nReconstructed ray direction from pixel:", ray_dir)
    
    hit3D, distance3D = ray_triangle_intersect3D(cam_pos, ray_dir, vertices[0], vertices[1], vertices[2])
    print("\n3D Intersection:")
    print("  Hit:", hit3D, "Distance:", distance3D)

    # --- Compare the distances ---
    if hit2D and hit3D:
        diff = abs(distance2D - distance3D)
        print("\nDifference between 2D and 3D intersection distances:", diff)
    else:
        print("\nOne or both intersections failed.")


2D triangle vertices (pixel coordinates and inv_distance):
[[ 0.2  0.2  0.2]
 [-0.2  0.2  0.2]
 [ 0.  -0.2  0.2]]

Test pixel (barycenter in 2D): [0.         0.06666667]

2D Intersection:


NameError: name 'distance2D' is not defined

In [ ]:
import numpy as np

def ray_tetrahedron_intersect_fused(orig, direction, v0, v1, v2, v3):
    """
    Replicates the ray-tetrahedron intersection test using plane clipping.
    Parameters:
        orig (np.ndarray)       : Ray origin, shape (3,)
        direction (np.ndarray)  : Ray direction, shape (3,)
        v0, v1, v2, v3 (np.ndarray): Tetrahedron vertices, each shape (3,)
    Returns:
        (intersect_bool, t_range)
        intersect_bool (bool) : True if intersection occurs, False otherwise
        t_range (tuple)       : (t_enter, t_exit) if intersect_bool is True; otherwise None
    """

    eps = 1e-10
    # Helper function for cross and dot to make code clearer
    def dot(a, b):   return np.dot(a, b)
    def cross(a, b): return np.cross(a, b)

    # --------------------------------------------------------------------
    # 1. Compute plane equations for the 4 faces (outward normals).
    #    Each face has normal n[i], and plane offset d[i].
    # --------------------------------------------------------------------

    # Face 0: (v0, v1, v2)
    n0 = cross(v1 - v0, v2 - v0)
    if dot(n0, v3 - v0) > 0.0:
        n0 = -n0
    d0 = -dot(n0, v0)

    # Face 1: (v0, v1, v3)
    n1 = cross(v1 - v0, v3 - v0)
    if dot(n1, v2 - v0) > 0.0:
        n1 = -n1
    d1 = -dot(n1, v0)

    # Face 2: (v0, v2, v3)
    n2 = cross(v2 - v0, v3 - v0)
    if dot(n2, v1 - v0) > 0.0:
        n2 = -n2
    d2 = -dot(n2, v0)

    # Face 3: (v1, v2, v3)
    n3 = cross(v2 - v1, v3 - v1)
    if dot(n3, v0 - v1) > 0.0:
        n3 = -n3
    d3 = -dot(n3, v1)

    # --------------------------------------------------------------------
    # 2. Initialize the intersection range [t_enter, t_exit]
    # --------------------------------------------------------------------
    t_enter = -1e30
    t_exit  =  1e30

    # --------------------------------------------------------------------
    # Clipping helper
    # --------------------------------------------------------------------
    def clip_plane(n, d):
        nonlocal t_enter, t_exit
        dist  = dot(n, orig) + d
        denom = dot(n, direction)

        # Ray is parallel?
        if abs(denom) < eps:
            # If the origin is outside the plane, no intersection
            if dist > 0.0:
                return False
        else:
            t_plane = -dist / denom
            if denom < 0.0:
                # entering
                if t_plane > t_enter:
                    t_enter = t_plane
            else:
                # exiting
                if t_plane < t_exit:
                    t_exit = t_plane
        return True

    # --------------------------------------------------------------------
    # 3..6. Clip against each plane
    # --------------------------------------------------------------------
    if not clip_plane(n0, d0): return (False, None)
    if not clip_plane(n1, d1): return (False, None)
    if not clip_plane(n2, d2): return (False, None)
    if not clip_plane(n3, d3): return (False, None)

    # Check if valid interval
    if t_enter > t_exit:
        return (False, None)

    # --------------------------------------------------------------------
    # 7. Final intersection range
    # --------------------------------------------------------------------
    if t_exit <= 0.0:
        return (False, None)

    t_enter = max(t_enter, 0.0)
    return (True, (t_enter, t_exit))

def pickTransformAxes(direction):
    """
    Pick the dominant axis (kz) where |dir[kz]| is largest, then pick kx, ky
    in a cyclic manner. Swap if dir[kz] < 0 to preserve winding.
    Returns a tuple (kx, ky, kz).
    """
    ad = np.abs(direction)
    kz = 0
    if ad[1] > ad[0] and ad[1] > ad[2]:
        kz = 1
    elif ad[2] > ad[0] and ad[2] > ad[1]:
        kz = 2

    kx = (kz + 1) % 3
    ky = (kx + 1) % 3

    # If the dominant component is negative, swap kx, ky
    if direction[kz] < 0.0:
        kx, ky = ky, kx

    return (kx, ky, kz)

def shearScalePoint(P, S, axes):
    """
    Shear+scale transform a 3D point P into the new space, assuming we have
    already translated so the ray origin is (0,0,0).
    P is an np.array of shape (3,)
    S is an np.array of shape (3,) for scale
    axes is a tuple (kx, ky, kz)
    """
    kx, ky, kz = axes
    # Output in new coordinates:
    #   x' = P[kx] - S.x * P[kz]
    #   y' = P[ky] - S.y * P[kz]
    #   z' = P[kz] * S.z
    return np.array([
        P[kx] - S[0] * P[kz],
        P[ky] - S[1] * P[kz],
        P[kz]  * S[2]
    ], dtype=float)

def cyrusBeckTetrahedronShearScale(orig, direction, v0, v1, v2, v3):
    """
    Alternative tetrahedron intersection test using the Cyrus-Beck approach
    in a transformed (sheared+scaled) space where the ray becomes (0,0,1).
    This function replicates the structure given in the original code.
    Parameters:
        orig (np.ndarray)       : Ray origin, shape (3,)
        direction (np.ndarray)  : Ray direction, shape (3,)
        v0, v1, v2, v3 (np.ndarray): Tetrahedron vertices, each shape (3,)
    Returns:
        (intersect_bool, t_range)
        intersect_bool (bool) : True or False. In the source snippet it always returns false,
                                so you might want to adjust if needed.
        t_range (tuple)       : (t_enter, t_exit) param range
    """

    eps = 1e-12

    # Face index array: each face is a triple of vertex indices, plus one "opposite" index.
    face_indices = [
        [3, 2, 1, 0],  # Face 0: (v3, v2, v1)
        [2, 3, 0, 1],  # Face 1: (v2, v3, v0)
        [1, 0, 3, 2],  # Face 2: (v1, v0, v3)
        [0, 1, 2, 3],  # Face 3: (v0, v1, v2)
    ]

    # A) Translate vertices so orig is the new (0,0,0)
    V0 = v0 - orig
    V1 = v1 - orig
    V2 = v2 - orig
    V3 = v3 - orig

    # B) Determine dominant axis and compute shear+scale
    axes = pickTransformAxes(direction)
    kx, ky, kz = axes

    # S: Shear/scale factors
    S = np.array([
        direction[kx] / direction[kz],
        direction[ky] / direction[kz],
        1.0 / direction[kz]
    ], dtype=float)

    # Transform each vertex into new space
    A0 = shearScalePoint(V0, S, axes)
    A1 = shearScalePoint(V1, S, axes)
    A2 = shearScalePoint(V2, S, axes)
    A3 = shearScalePoint(V3, S, axes)

    # In this new space, the ray is from (0,0,0) in direction (0,0,1).
    # We'll do standard Cyrus–Beck plane clipping.

    t_enter = 0.0
    t_exit = 1e30

    tverts = [A0, A1, A2, A3]

    def dot(a, b):   return np.dot(a, b)
    def cross(a, b): return np.cross(a, b)

    for face in face_indices:
        P0, P1, P2, P3 = [tverts[idx] for idx in face]

        # Face normal in new space; flip if necessary
        n = cross(P1 - P0, P2 - P0)
        if dot(n, P3 - P0) > 0.0:
            n = -n

        d = -dot(n, P0)
        dist = d  # Because origin is (0,0,0) => dot(n, origin) + d => d
        denom = n[2]  # dot(n, (0,0,1)) in new space

        if abs(denom) < eps:
            if dist > 0.0:
                return (False, None)
        else:
            t_plane = -dist / denom
            if denom < 0.0:
                if t_plane > t_enter:
                    t_enter = t_plane
            else:
                if t_plane < t_exit:
                    t_exit = t_plane

    if t_enter > t_exit:
        return (False, None)

    # Clip t_enter to be >= 0
    t_enter = max(0.0, t_enter)

    if t_exit <= 0.0:
        return (False, None)

    # The code snippet always returns False at the end. Adjust if needed.
    # For now, we replicate the original logic literally:
    return (True, (t_enter, t_exit))

cam_pos = np.array([0.0, 0.0, 0.0], dtype=np.float32)
viewmat = np.eye(4, dtype=np.float32)  # Identity view matrix.
projection_matrix = np.eye(3, dtype=np.float32)  # Identity projection (i.e. no additional mapping).

# Define a 3D triangle (in front of the camera at z = 5).
vertices = np.array([
    [1.0,  1.0,  5.0],
    [-1.0, 1.0,  2.0],
    [0.0,  -1.0, 3.0],
    [0.0,  -0.5, 4.0]
], dtype=np.float32)

# Convert the 3D triangle to a 2D representation.
pts_2d = point2image(vertices, viewmat, projection_matrix, cam_pos)
# pts_2d is (3, 3): each row is [x, y, inv_distance]

print("2D triangle vertices (pixel coordinates and inv_distance):")
print(pts_2d)

# Choose a test pixel: for example, the barycenter of the 2D triangle.
pixel_pos = np.mean(pts_2d[:, :2], axis=0)
print("\nTest pixel (barycenter in 2D):", pixel_pos)

# --- 3D Intersection ---
# In our simplified setup, the conversion from 3D to 2D is:
#   pixel = [x/z, y/z]
# so we can reconstruct a ray direction as [pixel_x, pixel_y, 1] (then normalize).
ray_dir = np.array([pixel_pos[0], pixel_pos[1], 1.0], dtype=np.float32)
ray_dir /= np.linalg.norm(ray_dir)
print("\nReconstructed ray direction from pixel:", ray_dir)


orig = cam_pos
direction = ray_dir
v0 = vertices[0]
v1 = vertices[1]
v2 = vertices[2]
v3 = vertices[3]
print(cyrusBeckTetrahedronShearScale(orig, direction, v0, v1, v2, v3))
print(ray_tetrahedron_intersect_fused(orig, direction, v0, v1, v2, v3))

2D triangle vertices (pixel coordinates and inv_distance):
[[ 0.2         0.2         0.2       ]
 [-0.5         0.5         0.5       ]
 [ 0.         -0.33333334  0.33333334]
 [ 0.         -0.125       0.25      ]]

Test pixel (barycenter in 2D): [-0.075       0.06041666]

Reconstructed ray direction from pixel: [-0.07465459  0.06013841  0.99539447]
(True, (2.975259348669417, 3.400085931375171))
(True, (2.9752593, 3.400086))
